# FloodNet Sensor Data Animation

This notebook demonstrates using the FloodNet client to query and visualize sensor data during Hurricane Ida in Brooklyn.

In [ ]:
# Install required packages
!pip install floodnet-client[spatial] matplotlib

In [9]:
import logging
from datetime import datetime
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from floodnet_client import SpatialFloodNetClient

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

In [10]:
# Initialize spatial client
client = SpatialFloodNetClient()

# Load Brooklyn boundary
brooklyn = gpd.read_file("data/test_data.gpkg", layer="brooklyn")
brooklyn_geom = brooklyn.geometry.iloc[0]

# Define Ida time period (Sep 1-2, 2021)
ida_start = datetime(2021, 9, 1, 0, 0)
ida_end = datetime(2021, 9, 2, 23, 59)

# Query depth data
logger.info("Querying depth data during Hurricane Ida...")
depth_data = client.get_depth_data_within_geometry(
    start_time=ida_start,
    end_time=ida_end,
    geometry=brooklyn_geom
)

if len(depth_data) > 0:
    logger.info(f"Found {len(depth_data)} depth readings")
else:
    logger.info("No depth readings found for this time period")

2025-01-22 21:23:55,341 - __main__ - INFO - Querying depth data during Hurricane Ida...
2025-01-22 21:23:55,361 - floodnet_client.client - INFO - Fetching deployment data from API
2025-01-22 21:23:55,581 - floodnet_client.client - INFO - Processed 233 deployment records
2025-01-22 21:23:55,624 - floodnet_client.spatial - INFO - Found 74 deployments within geometry
2025-01-22 21:23:55,624 - floodnet_client.spatial - INFO - Querying depth data from 2021-09-01 00:00:00 to 2021-09-02 23:59:00
2025-01-22 21:23:55,625 - floodnet_client.client - INFO - Fetching depth data for 74 deployments
2025-01-22 21:23:57,916 - floodnet_client.client - INFO - Got 510 valid readings for deployment widely_proud_lizard
2025-01-22 21:24:00,187 - floodnet_client.client - INFO - Got 664 valid readings for deployment daily_new_falcon
2025-01-22 21:24:01,620 - floodnet_client.client - INFO - Got 542 valid readings for deployment weekly_poetic_guinea
2025-01-22 21:24:05,513 - floodnet_client.client - INFO - Retri

In [11]:
# Prepare data for animation
depth_data['time'] = pd.to_datetime(depth_data['time'])
depth_data['hour'] = depth_data['time'].dt.floor('H')

# Get unique hours
hours = depth_data['hour'].unique()
hours.sort()

# Create base map
fig, ax = plt.subplots(figsize=(10, 10))
brooklyn.plot(ax=ax, color='lightgray', edgecolor='black')

# Initialize scatter plot
scat = ax.scatter([], [], c=[], cmap='viridis', 
                 vmin=0, vmax=depth_data['depth_mm'].max(),
                 s=100, alpha=0.8)

# Add colorbar
cbar = plt.colorbar(scat, ax=ax)
cbar.set_label('Water Depth (mm)')

# Add title
title = ax.set_title('')

def update(hour):
    """Update function for animation"""
    # Filter data for current hour
    current_data = depth_data[depth_data['hour'] == hour]
    
    # Update scatter plot
    scat.set_offsets(current_data[['geometry.x', 'geometry.y']])
    scat.set_array(current_data['depth_mm'])
    
    # Update title
    title.set_text(f'Hurricane Ida - {hour.strftime("%Y-%m-%d %H:%M")}')
    
    return scat, title

# Create animation
ani = FuncAnimation(fig, update, frames=hours, interval=200, blit=True)

# Display animation
plt.close()  # Prevents double display in notebook
from IPython.display import HTML
HTML(ani.to_jshtml())

/tmp/ipykernel_27462/1202164597.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  depth_data['hour'] = depth_data['time'].dt.floor('H')


AttributeError: 'DatetimeArray' object has no attribute 'sort'

In [ ]:
# Save animation as gif
ani.save('ida_brooklyn_animation.gif', writer='pillow', fps=2)
logger.info("Animation saved as ida_brooklyn_animation.gif")